Analyse DATA generated with COLMAP.

In [2]:
import numpy as np
import os
from glob import glob
import plotly.graph_objects as go

# `points3D.txt`
Each line in this file contains:

POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)

We mainly care about x,y,z,r,g,b here.

In [27]:
colmap_path = '/home/mauro/Documents/BlenderProjects/Reflective_3DGS/data'

pt_cld_path = os.path.join(colmap_path, 'sparse', '0_text', 'points3D.txt')
pt_cld = []

with open(pt_cld_path, 'r') as f:
    for idx, line in enumerate(f.readlines()):
        # Skip first lines
        if idx < 3:
            continue
        line = line.strip().split(' ')
        
        # Convert x,y,z,r,g,b values from str to float
        for i in range(1, 7):
            line[i] = float(line[i])
        
        
        xyz = [line[1], line[2], line[3]]
        rgb = [line[4]/255.0, line[5]/255.0, line[6]/255.0]
        seg = [0.0]   # Always static for now 
        
        pt_cld.extend([xyz + rgb + seg])
        
pt_cld = np.array(pt_cld)


In [28]:
fig = go.Figure(
    [
        go.Scatter3d(x=pt_cld[:, 0], y=pt_cld[:, 1],z=pt_cld[:, 2], mode='markers', marker=dict(size=2, color=pt_cld[:, 3:6])),
    ]
)
fig.show()

# Database
COLMAP stores information in its database. Because I have extracted data using the `SIMPLE_PINHOLE` model, the database stores information as:
```
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, F, CX, CY
```
and specifically (for example):
```
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
# Number of cameras: 1
1 SIMPLE_PINHOLE 600 400 857.73334393118137 300 200
```

Because it is a simple_pinhole camera, `Fx` and `Fy` are the same.

To reconstruct the K matrix:

In [9]:
def get_intrinsics_from_txt(path):
    """Convert the file `cameras.txt` extracted from colmap (SIMPLE_PINHOLE) to camera intrinsics."""
    ks = []
    with open(path, 'r') as f:
        for idx, line in enumerate(f.readlines()):
            # Skip first lines
            if idx < 3:
                continue
            line = line.strip().split(' ')

            # Convert x,y,z,r,g,b values from str to float
            for i in range(2, 7):
                line[i] = float(line[i])

            w = line[2]
            h = line[3]
            fx = line[4]
            fy = line[4]
            cx = line[5]
            cy = line[6]

            k = [ [fx, 0, cx], [0, fy, cy], [0, 0, 1] ]
            ks.append(k)
    
    return ks

In [13]:
path = '/home/mauro/Documents/BlenderProjects/Reflective_3DGS/data/sparse/0_text/cameras.txt'
ks = get_intrinsics_from_txt(path)
print(np.array(ks).shape)

(1, 3, 3)


To reconstruct w2c:

In [ ]:
def quaternion_to_rotation(q0, q1, q2, q3):
    # w, x, y ,z
    """
    Convert a quaternion into a full three-dimensional rotation matrix.

    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3)

    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix.
             This rotation matrix converts a point in the local reference
             frame to a point in the global reference frame.
    """
    # Extract the values from Q

    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)

    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)

    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1

    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])

    return rot_matrix


def get_w2c_from_text(path):
    # The values are taken from images.txt
    
    
    # Assuming R is the 3x3 rotation matrix and t is the 3x1 translation vector
    R = ...  # Your rotation matrix
    t = ...  # Your translation vector

    # Create the 4x4 transformation matrix
    W2C = np.eye(4)
    W2C[:3, :3] = R
    W2C[:3, 3] = -np.dot(R, t)
